## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('data.csv')

data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

- Столбцы <b>days_employed</b> и <b>total_income</b> имеют пропущенные значения, причем их количество совпадает
- Столбец <b>days_employed</b> содержит отрицательные значения. Это поле также имеет дробные значения, что недопустимо для стажа.
- Значение в <b>столбце education</b> записаны в разных регистрах
- Значение в столбцах указаны на русском языке, однако поле gender содержит английские аббревиатуры 

### Шаг 2. Предобработка данных

### Обработка пропусков

In [3]:
data['days_employed'].isna().sum()

2174

In [4]:
data['total_income'].isna().sum()

2174

In [5]:
data[data['days_employed'].isna()]['total_income'].isna().sum()

2174

Пустые значение в обоих полях совпадают

In [6]:
data[data['days_employed'].isna()].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


Других закономерностей не обнаружено

In [7]:
def present(count_100_percent, count__x_percent):
    return 100/ (count_100_percent / count__x_percent)

In [8]:
present(len(data), data['days_employed'].isna().sum())

10.099883855981417

Пропущено 10% данных, довольно большой процент. Пропуски необходимо заполнить предварительно изучив и исправив значения в столбцах.

In [9]:
present(len(data), data[ data['days_employed'] < 0]['days_employed'].count())

73.89547038327527

Почти все значения отрицательные, скорее всего, ошибка заполнения.

In [10]:
data['days_employed'] = abs(data['days_employed'])

In [11]:
data.loc[data['days_employed'].notna(),'days_employed'] = data[data['days_employed'].notna()]['days_employed'].astype('int')

In [12]:
data['max_days_employed'] = (data['dob_years'] - 12) * 365.25 
def count_present_live_less_than_job(): 
    live_less_than_job = data.loc[data['max_days_employed'] < data['days_employed']]['days_employed'].count()
    return present(len(data), live_less_than_job)

In [13]:
count_present_live_less_than_job()

16.362369337979093

По закону, люди могут работать только с 12 лет. Трудовой стаж больше периода, в течении которого, человек может работать - ошибка. Таких 16.3 процента.

In [14]:
data.drop(data.loc[data['max_days_employed'] < data['days_employed']].index,inplace=True)
count_present_live_less_than_job()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in long_scalars
  


0.0

In [15]:
data['days_employed'] = data['days_employed'].fillna(data['days_employed'].median())
count_present_live_less_than_job()

0.05554629783924902

In [16]:
data.drop(data.loc[data['max_days_employed'] < data['days_employed']].index,inplace=True)
count_present_live_less_than_job()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in long_scalars
  


0.0

#### Поле total_income

In [17]:
mean_by_income_type = data[data['total_income'].notna()].groupby('income_type')['total_income'].mean()

In [18]:
mean_by_income_type

income_type
в декрете           53829.130729
госслужащий        171008.225815
компаньон          202483.482019
предприниматель    499163.144947
сотрудник          161390.227204
студент             98201.625314
Name: total_income, dtype: float64

In [19]:
def fill_total_income(row):
    if row['total_income'] != float('nan'):
        return row['total_income']
    
    return mean_by_income_type.loc[mean_by_income_type['income_type'] == row['income_type'],'total_income']

In [20]:
data['total_income'] = data.apply(fill_total_income, axis=1)

In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17993 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   children           17993 non-null  int64  
 1   days_employed      17993 non-null  float64
 2   dob_years          17993 non-null  int64  
 3   education          17993 non-null  object 
 4   education_id       17993 non-null  int64  
 5   family_status      17993 non-null  object 
 6   family_status_id   17993 non-null  int64  
 7   gender             17993 non-null  object 
 8   income_type        17993 non-null  object 
 9   debt               17993 non-null  int64  
 10  total_income       15829 non-null  float64
 11  purpose            17993 non-null  object 
 12  max_days_employed  17993 non-null  float64
dtypes: float64(3), int64(5), object(5)
memory usage: 1.9+ MB


### Вывод

Два поля содержали пустые значения (<b>days_employed</b> и <b>total_income</b>). Поле <b>days_employe</b> содержало записи, где трудовой стаж превышал период, в течении которого, человек может работать, такие данные были удалены. Пустые значения заменены на медиану, так — как трудовой стаж в днях — это переменная целого типа, тогда как средние может быть дробным (считая вещественную часть ошибкой при выгрузки данных). Для замены пустых значений в поле  <b>total_income</b>, данные были сгруппированы по полю <b>income_type</b> так, как средняя зарплата студента существенно отличается от средней предпринимателя. Пустые значения заменены  средним из соответствующей категории.

### Замена типа данных

### Вывод

### Обработка дубликатов

### Вывод

### Лемматизация

### Вывод

### Категоризация данных

### Вывод

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

### Вывод

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

### Вывод

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

### Вывод

- Как разные цели кредита влияют на его возврат в срок?

### Вывод

### Шаг 4. Общий вывод

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [ ]  файл изучен;
- [ ]  определены пропущенные значения;
- [ ]  заполнены пропущенные значения;
- [ ]  есть пояснение, какие пропущенные значения обнаружены;
- [ ]  описаны возможные причины появления пропусков в данных;
- [ ]  объяснено, по какому принципу заполнены пропуски;
- [ ]  заменен вещественный тип данных на целочисленный;
- [ ]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [ ]  удалены дубликаты;
- [ ]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [ ]  описаны возможные причины появления дубликатов в данных;
- [ ]  выделены леммы в значениях столбца с целями получения кредита;
- [ ]  описан процесс лемматизации;
- [ ]  данные категоризированы;
- [ ]  есть объяснение принципа категоризации данных;
- [ ]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [ ]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [ ]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [ ]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [ ]  в каждом этапе есть выводы;
- [ ]  есть общий вывод.